In [122]:
def read_input(file_str: str):
    with open(file_str) as f:

        lines = f.readlines()
        lines.reverse()

        n = int(lines.pop().split()[0])
        parents = [int(s) for s in lines.pop().split(' ')]
        values = [int(s) for s in lines.pop().split(' ')]

        diseases = []
        num_of_diseases = int(lines.pop().split()[0])
        for i in range(num_of_diseases):
            dis = lines.pop()
            diseases.append([int(s) for s in dis.split(' ')])

        patients = []
        num_of_patients = int(lines.pop().split()[0])
        for i in range(num_of_patients):
            dis = lines.pop()
            patients.append([int(s) for s in dis.split(' ')])

        # return [n, relationships, values, diseases, patients]
        return {
            "n": n,
            "parents": parents,
            "values": values,
            "diseases": diseases,
            "patients": patients,
        }

In [123]:
data = read_input('input.txt')
n = data["n"]
parents = data["parents"]
values = data["values"]
diseases = data["diseases"]
patients = data["patients"]
print(n)
print(parents)
print(values)
print(diseases)
print(patients)

10
[1, 1, 3, 3, 4, 4, 5, 5, 5]
[5, 7, 8, 13, 18, 14, 15, 21, 20, 29]
[[2, 4, 2], [1, 10]]
[[3, 5, 9, 8], [1, 6], [2, 7, 10], [1, 10]]


In [105]:
class Node(object):
    def __init__(self, index, value):
        self.index = index
        self.value = value
        self.children = []
        self.parent = None
        self.subtree = []

    def add_child(self, index):
        self.children.append(index)

    def add_parent(self, index):
        self.parent = index

    def init_subtree(self, subtree_list: list):
        self.subtree = subtree_list

In [ ]:
for i in range(n):
    node = Node(i, values[i])
    if i != 0:
        node.add_parent(parents[i])

In [142]:
from collections import defaultdict

parents_hmap = defaultdict(int)  # child: parent
children_hmap = defaultdict(list)  # parent: [children]
values_hmap = defaultdict(int)  # node: value
is_checked_hmap = defaultdict(bool)  # node: value
subtrree_hmap = defaultdict(list)  # parent: [all children in subtree]

for i in range(len(parents)):
    child_index = i + 2
    parent_index = parents[i]

    parents_hmap[child_index] = parent_index
    children_hmap[parent_index].append(child_index)
    values_hmap[i] = values[i]
    is_checked_hmap[i] = False

values_hmap[n - 1] = values[n - 1]

print(parents_hmap)
print(children_hmap)
print(values_hmap)

NameError: name 'Fal' is not defined